In [ ]:
# !sudo pip install transformers peft evaluate seqeval

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import (
    get_peft_config,
    PeftModel,
    PeftConfig,
    get_peft_model,
    LoraConfig,
    TaskType,
)
import evaluate
import torch
import numpy as np

model_checkpoint = "roberta-large"
model_checkpoint = 'dccuchile/bert-base-spanish-wwm-cased'
lr = 1e-3
batch_size = 8
num_epochs = 10

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
import pandas as pd
from more_itertools import split_at
from glob import glob

data = pd.DataFrame()
for path in glob('output/*.csv'):
    df = pd.read_csv(path)
    df['path'] = path

    data = pd.concat([df, data], ignore_index=False)

cat_count = dict(data['label'].value_counts())
categories = {v: k for k, v in enumerate(data['label'].unique())}
categories = {k: v for k, v in categories.items() if cat_count.get(k, 0) > 3}

data['label'] = data['label'].map(categories)

In [ ]:
cat_count

In [ ]:
import numpy as np
from more_itertools import unzip 
from tqdm.auto import tqdm

asd = []
for path, group in tqdm(data.groupby('path')):
    indices, rows = unzip(group.iterrows())
    tuples = map(lambda row: (row['original'], row['label']), rows)
    tuples = split_at(tuples, lambda x: pd.isna(x[1]))

    for paragraph in tuples:
        df = pd.DataFrame(paragraph)
        # if not 3 <= len(df) <= 10:
        #     continue
        if len(df) < 3:
            continue
        tokens = df[0].values
        labels = df[1].values.astype(int)
        nlabels = np.sum(labels)

        if nlabels == 0 and np.random.rand() < 0.99:
            # print('skipping no entity paragraph')
            continue

        if len(tokens) != len(labels):
            print('mismatch size')
            continue
        
        asd.append(
            {
                'n_labels': np.sum(labels),
                'tokens': list(tokens),
                'tags': labels
            }
        )

len(asd)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

train, test = train_test_split(asd, test_size=0.2, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)

In [ ]:
dataset = DatasetDict({
    'train': Dataset.from_list(train),
    'validation': Dataset.from_list(val),
    'test': Dataset.from_list(test),
})
dataset

In [ ]:
df.value_counts().loc[0]

In [ ]:
import seaborn as sns

df = pd.DataFrame(asd)['n_labels']
sns.countplot(df)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
len(tokenized_dataset['train'])

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
code2label = {v: k for k, v in categories.items()}
code2label

In [ ]:
dataset['train'][10]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(categories.keys()), id2label=code2label, label2id=categories
)

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=16, lora_alpha=16, lora_dropout=0.1, bias="all"
)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
text = "El imputado Ramiro Ramallo Martinez dni 88.384.425"
inputs = tokenizer(text, return_tensors="pt")
model.to('cpu')
with torch.no_grad():
    logits = trainer.model(**inputs).logits

tokens = inputs.tokens()

predictions = torch.argmax(logits, dim=2)

for token, prediction in zip(tokens, predictions[0].numpy()):
    print((token, model.config.id2label[prediction]))

In [ ]:
training_args = TrainingArguments(
    output_dir="roberta-large-lora-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()